<a href="https://colab.research.google.com/github/erazocar/LSI_machinelearnhw2/blob/main/LSI_VSfinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing required libraries
import numpy as np
import os

#Gensim library for document selection and LSI
import gensim as gn

In [ ]:
#Reader for each file, appended to a list.
def reader(path):
  d=[]
  col = []
  filenames = os.listdir(path)
  for file in filenames:
    if file != '.DS_Store':
      filepath = path +'/' + file
      col.append(file)
      with open(filepath, 'r') as infile:
        content = infile.read().split('\n')
        d.append(content)
      for i in range(len(d)):
        d[i] = list(filter(None, d[i]))
        d[i] = np.array(d[i], dtype=str)
  print("Total number of documents: ", len(d))
  return d, col

In [ ]:
#Create a gensim dictionary and corpus
def corpus(doc_clean):
  dictionary = gn.corpora.Dictionary(doc_clean)
  doc_term_matrix=[dictionary.doc2bow(doc) for doc in doc_clean]
  return dictionary, doc_term_matrix

In [ ]:
#Create and define the LSI model from gensim
def gensim_lsi(doc_clean, num_concepts, words):
  dictionary, doc_term_matrix = corpus(doc_clean)
  lsimodel = gn.models.LsiModel(doc_term_matrix, num_topics=num_concepts, id2word=dictionary)
  V = gn.matutils.corpus2dense(lsimodel[doc_term_matrix], len(lsimodel.projection.s)).T / lsimodel.projection.s
  #topics = lsimodel.show_topics(num_topics=num_concepts, num_words = words)
  my_dict = {'Topic_'+str(i): [token for token, score in lsimodel.show_topic(i, topn=words)] for i in range(0, lsimodel.num_topics)}
  return lsimodel, V, dictionary, doc_term_matrix, my_dict

In [ ]:
#Read paths and create contents for the documents.
#Please put the local paths were the queries and docs are located when running.
path = '/content/drive/MyDrive/MachineLearning/homework-1/docs'

#Declare variables for both the queries and docs and the corresponding names.
docs, docs_names = reader(path)

Total number of documents:  500


In [ ]:
#Create the LSI model and obtain the results for each document.
concepts = 5
words = 10
LSI, V, dic, term_matrx, topics_words = gensim_lsi(docs, concepts, words)
topics_words

/usr/local/lib/python3.7/dist-packages/gensim/matutils.py:502: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  result = np.column_stack(sparse2full(doc, num_terms) for doc in corpus)


{'Topic_0': ['probes',
  'surface',
  'moon',
  'venus',
  'mariner',
  'images',
  'solar',
  'spacecraft',
  'soviet',
  'pioneer'],
 'Topic_1': ['slip',
  'packet',
  'phone',
  'echo',
  'goto',
  'surface',
  'probes',
  'dial',
  'pktmux',
  'c/slipper'],
 'Topic_2': ['religious',
  'religion',
  'belief',
  'slip',
  'exist',
  'atheist',
  'beliefs',
  'religions',
  'sort',
  'exists'],
 'Topic_3': ['telescope',
  'surface',
  'probes',
  'propulsion',
  'slip',
  'astronomical',
  'moon',
  'mariner',
  'observatory',
  'religious'],
 'Topic_4': ['printer',
  'deskjet',
  'canon',
  'quality',
  'print',
  'printers',
  'bubblejet',
  'paper',
  'bought',
  'printing']}

In [ ]:
#Obtain the 5 top singular values from the SVD central matrix.
LSI.projection.s

array([110.65142994,  93.4225172 ,  88.96466966,  79.47344215,
        69.15084486])

In [ ]:
#Create a list of queries derived from the top 5 concepts from the documents.
queries=[]
for i in topics_words:
  queries.append(topics_words[i])
print(queries)

[['probes', 'surface', 'moon', 'venus', 'mariner', 'images', 'solar', 'spacecraft', 'soviet', 'pioneer'], ['slip', 'packet', 'phone', 'echo', 'goto', 'surface', 'probes', 'dial', 'pktmux', 'c/slipper'], ['religious', 'religion', 'belief', 'slip', 'exist', 'atheist', 'beliefs', 'religions', 'sort', 'exists'], ['telescope', 'surface', 'probes', 'propulsion', 'slip', 'astronomical', 'moon', 'mariner', 'observatory', 'religious'], ['printer', 'deskjet', 'canon', 'quality', 'print', 'printers', 'bubblejet', 'paper', 'bought', 'printing']]


In [ ]:
#Define a similarity model from gensim
tf_idf = gn.models.TfidfModel(term_matrx)

#Please put the location were the model from gensim can be saved.
path2 = '/content/drive/MyDrive/MachineLearning/homework-2/'

#crete model for cosine similarity in local directory with the dictionary from
#extracted from the documents
sims2 = gn.similarities.Similarity(path2, tf_idf[term_matrx], num_features=len(dic))

In [ ]:
#Store arrays with both the values and index for highest 10 scoring documents. 
result_docs = []
finalres = []

#Loop through each of the queries based and calculate cosine similarity.
for query in queries:
  query_doc_bow = dic.doc2bow(query)
  query_doc_tf_idf = tf_idf[query_doc_bow]
  myres = sims2[query_doc_tf_idf]
  result_docs.append((-myres).argsort()[:10])
  finalres.append(myres)

/usr/local/lib/python3.7/dist-packages/gensim/similarities/docsim.py:518: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  result = numpy.hstack(shard_results)


In [ ]:
#Store values for highest 10 rated documents.
value =[]

#Obtain the values for similarities for each query per the highest 10t queries.
for j in range(len(finalres)):
  a = []
  for k in range(len(result_docs)):
    a.append(finalres[j][result_docs[j][k]])
  value.append(a)

print(value)

[[0.6271928, 0.2096867, 0.1765453, 0.16844536, 0.14709482], [0.6558138, 0.240949, 0.13982326, 0.055072147, 0.05436644], [0.58607084, 0.27872834, 0.22796525, 0.18467978, 0.18037388], [0.3729824, 0.23799735, 0.21445556, 0.17928259, 0.16968456], [0.7916511, 0.31970146, 0.31653395, 0.25521526, 0.21933538]]


In [ ]:
#Store the names of the final docs with 10th highest scores.
finaldocs=[]

#Loop through dictionary and obtain the names based on the name list.
for j in range(len(result_docs)):
  b=[]
  for k in result_docs[j]:
    b.append(docs_names[k])
  finaldocs.append(b)

print(finaldocs)

[['59905', '59871', '59909', '60210', '60170', '60195', '59873', '59874', '59848', '60187'], ['10011', '10078', '59905', '59871', '59909', '60208', '101578', '60198', '60200', '101586'], ['51060', '51153', '10078', '51161', '10011', '49960', '51135', '51164', '51179', '51130'], ['59905', '60103', '10078', '59871', '10011', '59874', '60170', '60208', '60210', '59909'], ['10034', '10044', '10085', '10092', '10072', '10100', '10047', '10009', '10071', '10032']]
